In [1]:
import torch
from transformers import *

In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-large-cased")
#model = BertModel.from_pretrained("bert-base-uncased")
lm = BertForMaskedLM.from_pretrained("bert-large-cased")
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
#lm = GPT2LMHeadModel.from_pretrained('gpt2-medium')

In [3]:
#model.eval()
lm.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [4]:
#model.to("cuda")
lm.to("cuda")

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [307]:
lm_states = [None] * 3

In [315]:
tokens = tokenizer.tokenize("[SEP] Although he had already eaten a large meal, he was still very [MASK] [MASK] [MASK] [MASK] [MASK]")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
tokens_tensor = torch.tensor([token_ids, token_ids]).to("cuda")
#tokens_tensor
#token_ids

In [316]:
segment_ids = [0] * len(tokens)
segments_tensor = torch.tensor([segment_ids]).to("cuda")
#segments_tensor

In [317]:
with torch.no_grad():
    lm_states[2] = lm(tokens_tensor, token_type_ids=segments_tensor)

In [351]:
#t1 = torch.tensor([0.1, 0.2, 0.3])
#t2 = torch.tensor([0.2, 0.3, 0.4])
#t3 = torch.tensor([0.1, 0.3, 0.5])
#ts = torch.stack([t1, t2, t3])
#torch.sum(ts, dim=0)
outs = torch.sum(torch.stack([lm_states[0][0][0][-3], lm_states[1][0][0][-4], lm_states[2][0][0][-5]]), dim=0)
#lm_states[2][0][0][-5]
#outs.size()

torch.Size([28996])

In [352]:
p = torch.topk(torch.softmax(outs, 0, torch.float32), 10)
p

torch.return_types.topk(
values=tensor([0.4769, 0.1966, 0.1430, 0.0579, 0.0512, 0.0267, 0.0155, 0.0062, 0.0049,
        0.0043], device='cuda:0'),
indices=tensor([1277, 7215, 7555, 4264, 8504, 5737, 4871, 3888, 7229, 9333],
       device='cuda:0'))

In [353]:
for t in zip(tokenizer.convert_ids_to_tokens([i.item() for i in p.indices]), [v.item() for v in p.values]):
    print(t)

('much', 0.47693562507629395)
('excited', 0.19656753540039062)
('hungry', 0.14301860332489014)
('concerned', 0.05791427567601204)
('uncomfortable', 0.05117257684469223)
('upset', 0.026653718203306198)
('tired', 0.015524517744779587)
('interested', 0.006166622042655945)
('pleased', 0.004930931609123945)
('disappointed', 0.004304708447307348)


In [ ]:
'for', 0.14375771582126617

In [ ]:
3 MASK
('The', 0.22463715076446533)
('He', 0.08895274996757507)
('And', 0.06715645641088486)
('I', 0.04374244809150696)
('A', 0.03553885966539383)
('But', 0.027849610894918442)
('Then', 0.02589087188243866)
('You', 0.022679315879940987)
('His', 0.01970822736620903)
('Now', 0.016301993280649185)

In [ ]:
4 MASK
('The', 0.2075418382883072)
('And', 0.08329132199287415)
('He', 0.06717020273208618)
('Then', 0.058516643941402435)
('But', 0.0459866002202034)
('So', 0.045035168528556824)
('A', 0.032907549291849136)
('I', 0.03204396367073059)
('You', 0.02373039536178112)
('Now', 0.02367119863629341)

In [ ]:
5 MASK
('The', 0.13816286623477936)
('And', 0.08287341892719269)
('So', 0.07413381338119507)
('Then', 0.061999451369047165)
('But', 0.04996540769934654)
('He', 0.04658994451165199)
('Someone', 0.03473859652876854)
('"', 0.03286093473434448)
('I', 0.02899748831987381)
('A', 0.023754343390464783)

In [ ]:
6 MASK
('Someone', 0.09099213778972626)
('The', 0.08973071724176407)
('Then', 0.08499286323785782)
('And', 0.07965865731239319)
('So', 0.06291094422340393)
('He', 0.05035630986094475)
('But', 0.03429672122001648)
('I', 0.032312363386154175)
('"', 0.028058426454663277)
('A', 0.02554512582719326)

In [ ]:
7 MASK
('Someone', 0.103166863322258)
('And', 0.09235195815563202)
('The', 0.09034273028373718)
('Then', 0.07879479229450226)
('He', 0.05864255502820015)
('So', 0.0558164156973362)
('But', 0.03331487998366356)
('I', 0.03017641045153141)
('A', 0.02472865954041481)
('You', 0.022422486916184425)